In [127]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np

from tqdm import tqdm

# set env
np.set_printoptions(suppress=True)
pd.set_option('display.max_columns', 100)


from difflib import SequenceMatcher
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()



data_root = '../data/_datasets/cluster_res'
# 分类数据
pwtc_f = 'protein_wt_cluster.tsv'
ddna_f = 'dsDNA_80_cluster.txt'
drna_f = 'dsRNA_80_cluster.txt'
sdna_f = 'ssDNA_80_cluster.txt'
srna_f = 'ssRNA_80_cluster.txt'
# 主要数据
mainf = 'seq_dg_230620.txt'
mainf = 'seq_dg_230703.txt'
wtf = 'wt.tsv'


mainf = os.path.join(data_root, mainf)
madf = pd.read_csv(mainf, sep='\t', low_memory=False)
print(len(madf))
# madf.head(3)

madf['UniProt'] = madf['UniProt'].apply(lambda x: x.replace('A0A140NGK1','P0A6X3'))
madf['UniProt'] = madf['UniProt'].apply(lambda x: x.replace('C1IFD2','P0A6X3'))
madf['UniProt'] = madf['UniProt'].apply(lambda x: x.replace('Q71TA8','P24042'))
madf['UniProt'] = madf['UniProt'].apply(lambda x: x.replace('A0A0H2UPA7','Q8DQG2'))
madf['UniProt'] = madf['UniProt'].apply(lambda x: x.replace('C3SZN7','P0AGK8'))

# seq_dg_230620.txt 是大文件;
# dataset_new 是更新后，各个核酸复合物的数据集文件;
# cluster_new 是聚类文件,wt 数据集直接将 uniprot+na连接表示一个复合物，
# mut 将 uniprot+protein_mutation+na 表示复合物。
# 单双链的na不同，单链为 na_index列,双链为 newnafea_na_job 列;

# mutation_type
# UniProt
# nucleic_acid_type_new
# double: newnafea_na_job
# single: na_index


wtdf = pd.read_csv( os.path.join(data_root, wtf), sep='\t' )
print(len(wtdf))
# wtdf.head(3)
# protein_sequence nucleotide_sequence dG complex_id

wtdf['UniProt'] = wtdf['UniProt'].apply(lambda x: x.replace('A0A140NGK1','P0A6X3'))
wtdf['UniProt'] = wtdf['UniProt'].apply(lambda x: x.replace('C1IFD2','P0A6X3'))
wtdf['UniProt'] = wtdf['UniProt'].apply(lambda x: x.replace('Q71TA8','P24042'))
wtdf['UniProt'] = wtdf['UniProt'].apply(lambda x: x.replace('A0A0H2UPA7','Q8DQG2'))

seq2data = dict()
uniq_dlist = list()
for i in range(len(madf)):
    row = madf.loc[i]
    seq = row['protein_sequence'] + row['nucleotide_sequence']
    if seq not in seq2data:
        seq2data[seq] = 1
        uniq_dlist.append(True)
    else:
        uniq_dlist.append(False)

mask = pd.Series(uniq_dlist)
madf = madf[mask]
madf = madf.reset_index()
print("after remove, len =", len(madf))
# check valid
# for i in tqdm(range(len(wtdf))):
#     row = wtdf.loc[i]
#     key = row['protein_sequence'] + row['nucleotide_sequence']
#     cid = row['complex_id']
#     tmp = madf[madf['complex_id'] == cid]
#     key2 = tmp['protein_sequence'].iloc[0] + tmp['nucleotide_sequence'].iloc[0]
#     if key != key2:
#         print(cid)
#     if len(tmp) != 1:
#         print(cid)

17199
4846
after remove, len = 9057


In [8]:
# 相同蛋白序列但是UniProt ID 不同：
# P0A6X3,A0A140NGK1,C1IFD2
# P24042,Q71TA8
# Q8DQG2,A0A0H2UPA7
# P0AGK8 C3SZN7

# main_df['UniProt'] = main_df['UniProt'].apply(lambda x: x.replace('A0A140NGK1','P0A6X3'))
# main_df['UniProt'] = main_df['UniProt'].apply(lambda x: x.replace('C1IFD2','P0A6X3'))

# main_df['UniProt'] = main_df['UniProt'].apply(lambda x: x.replace('Q71TA8','P24042'))

# main_df['UniProt'] = main_df['UniProt'].apply(lambda x: x.replace('A0A0H2UPA7','Q8DQG2'))

# main_df['UniProt'] = main_df['UniProt'].apply(lambda x: x.replace('C3SZN7','P0AGK8'))

# data_dir = '../data/_datasets/'
# data_file = 'seq_dg_cluster_final_230607_v3.csv'
# data_file = os.path.join(data_dir, data_file)
# main_df.to_csv(data_file, index=False, sep='\t')


# madf[madf['mutation_type'] == 'WILD']

In [128]:
def createMap(data_root, fpath, prefix, output, cls):
    df = pd.read_csv( os.path.join(data_root, fpath), sep='\t')
    cls = df['cluster_index'].values
    for i in range(len(df)):
        row = df.loc[i]
        keys = row['cluster_member'].split(',')
        for k in keys:
            output[k] = prefix + row['cluster_index']

cid2wt = dict()
for c in madf.loc[:, 'complex_id'].values:
    cid2wt[c] = False

### 创建mapping
# cluster_index  cluster_number  cluster_member
prot_cls = None
unp2cls = dict()
createMap(data_root, pwtc_f, 'P', unp2cls, prot_cls)

ddna_cls, drna_cls, sdna_cls, srna_cls = None, None, None, None
ddna2cls, drna2cls, sdna2cls, srna2cls = dict(), dict(), dict(), dict()
createMap(data_root, ddna_f, 'DDNA', ddna2cls, ddna_cls)
createMap(data_root, drna_f, 'DRNA', drna2cls, drna_cls)
createMap(data_root, sdna_f, 'SDNA', sdna2cls, sdna_cls)
createMap(data_root, srna_f, 'SRNA', srna2cls, srna_cls)


### 处理wt数据
if 'pclass' not in wtdf.columns:
    wtdf.insert(len(wtdf.columns), 'pclass', 'None', allow_duplicates=False)
if 'nuclass' not in madf.columns:
    madf.insert(len(madf.columns), 'nuclass', 'None', allow_duplicates=False)
    
for i in range(len(wtdf)):
    row = wtdf.loc[i]
    if row['UniProt'] not in unp2cls:
        print("not find uniProt in wt data: ", i, row['complex_id'], row['UniProt'])
    else:
        wtdf.loc[i, 'pclass'] = unp2cls[ row['UniProt'] ]
    cid2wt[row['complex_id']] = True
    #
    nc_type = row['nucleic_acid_type_new']
    d_ncid = row['newnafea_na_job']
    s_ncid = row['na_index']
    if nc_type == 'dsDNA':
        wtdf.loc[i, 'nuclass'] = ddna2cls[ d_ncid ]
    elif nc_type == 'dsRNA':
        wtdf.loc[i, 'nuclass'] = drna2cls[ d_ncid ]
    elif nc_type == 'ssDNA':
        wtdf.loc[i, 'nuclass'] = sdna2cls[ s_ncid ]
    elif nc_type == 'ssRNA':
        wtdf.loc[i, 'nuclass'] = srna2cls[ s_ncid ]
    else:
        print("is not valid dna/rna type")

### 处理主数据
if 'wt_complex' not in madf.columns:
    madf.insert(len(madf.columns), 'wt_complex', False, allow_duplicates=False)
if 'pclass' not in madf.columns:
    madf.insert(len(madf.columns), 'pclass', 'None', allow_duplicates=False)
if 'nuclass' not in madf.columns:
    madf.insert(len(madf.columns), 'nuclass', 'None', allow_duplicates=False)

for i in range(len(madf)):
    row = madf.loc[i]
    if row['UniProt'] not in unp2cls:
        print("not find uniProt in ma data: ", i, row['complex_id'], row['UniProt'])
    else:
        madf.loc[i, 'pclass'] = unp2cls[ row['UniProt'] ]
    #
    madf.loc[i, 'wt_complex'] = cid2wt[ row['complex_id'] ]
    #
    nc_type = row['nucleic_acid_type_new']
    d_ncid = row['newnafea_na_job']
    s_ncid = row['na_index']
    if nc_type == 'dsDNA':
        madf.loc[i, 'nuclass'] = ddna2cls[ d_ncid ]
    elif nc_type == 'dsRNA':
        madf.loc[i, 'nuclass'] = drna2cls[ d_ncid ]
    elif nc_type == 'ssDNA':
        madf.loc[i, 'nuclass'] = sdna2cls[ s_ncid ]
    elif nc_type == 'ssRNA':
        madf.loc[i, 'nuclass'] = srna2cls[ s_ncid ]
    else:
        print("is not valid dna/rna type")

# mutation_type
# UniProt
# nucleic_acid_type_new
# double: newnafea_na_job
# single: na_index

madf['base_class'] = madf[['nuclass', 'pclass']].agg('-'.join, axis=1)
wtdf['base_class'] = wtdf[['nuclass', 'pclass']].agg('-'.join, axis=1)

print('done')

done


In [129]:
# gkey is str of list of str
def createGroup(gkey, indf, groups_by_gkey):
    for k, subdf in indf.groupby(gkey):
        cp = subdf.copy(deep=True)
        cp = cp.reset_index()
        print(k, len(cp))
        groups_by_gkey[k] = cp
#

# all_cls2ind = dict()
# all_cls_set = set(madf['base_class'])
# for i, c in enumerate(all_cls_set):
#     all_cls2ind[c] = i

# for k, subdf in madf.groupby(['pclass', 'nuclass']):
#     if len(subdf) > 1:
#         print(k, len(subdf))
groups_by_wt = dict()
for k, subdf in madf.groupby('wt_complex'):
    cp = subdf.copy(deep=True)
    cp = cp.reset_index()
    print(k, len(cp))
    groups_by_wt[k] = cp
    
wt_sub_df = groups_by_wt[True]
mut_sub_df = groups_by_wt[False]

all_cls_set = set(madf['base_class'].values)
wt_cls_set = set(wt_sub_df['base_class'].values)
mut_cls_set = set(mut_sub_df['base_class'].values)

# uniq_wt_cls_indices = set([all_cls2ind[c] for c in wt_cls_set])
# uniq_mut_cls_indices = set([all_cls2ind[c] for c in mut_cls_set])
# cls_indices_only_in_wt = uniq_wt_cls_indices - uniq_mut_cls_indices

# in WT but not in MUT
cls_only_in_wt = wt_cls_set - mut_cls_set

# len(all_cls_set) - len(mut_cls_set - wt_cls_set) - len(cls_only_in_wt)
### only in WT: 3207
### only in MUT: 207
### in Both: 808
### all: 4222


### map from base class to complex id
map_cls2cid = dict()
for k, subdf in madf.groupby('base_class'):
    map_cls2cid[k] = subdf['complex_id'].values
    
###

False 4219
True 4838


In [126]:
len(madf), len(ma_sim_df)

(9057, 17122)

In [136]:
g_prot_cls2ind = dict()
g_ddna_cls2ind = dict()
g_drna_cls2ind = dict()
g_sdna_cls2ind = dict()
g_srna_cls2ind = dict()

# def getClsIndex(cls_list):
#     return {x:i for i, x in enumerate(cls_list)}
    
# def getClsIndex(cls, mapping):
#     return mapping[cls]

# def getClsIndex(cls):
#     return int(cls.split('_')[1])


### protein_index   na_index

def getMeanSim(sim_mat, mapping2key, cids_a, cids_b):
    # cid -> protein_index -> sim_mat.index
    indices_a = np.array([mapping2key[c] for c in cids_a])
    indices_b = np.array([mapping2key[c] for c in cids_b])
#     print(indices_a)
#     print(indices_b)
    sim = sim_mat.loc[indices_a, indices_b] # (num_a, num_b)
    return sim.mean()
    

# def getNcMeanSim(sim_mat, mapping2key, cids_a, cids_b):
#     ## TODO: create prot2index
#     indices_a = np.array([getClsIndex(nc) for nc in cids_a])
#     indices_b = np.array([getClsIndex(nc) for nc in cids_b])
    
#     sim = sim_mat[indices_a, indices_b] # (num_a, num_b)
#     return sim.mean()

### input i and j are value from sim_mat
def computeDist(i, j):
    i += 1
    j += 1
    return i * j * (i + j) / (np.abs(i - j) + 1)

# The dataset "seq_dg_230702.txt" contains information about dG. 
# The "nucleic_acid_type_new" column represents the four types of nucleic acids. 
# The "dG_ori" column contains the original dG values obtained from the literature. 
# The "complex_type" column indicates whether the complex is of the wild type or mutant type, 
# allowing us to divide the dataset based on this information. 
# By the way,If the value in the "complex_type" column is "dele," 
# it indicates that the complex needs to be removed from the dataset.

ma_sim_f = 'seq_dg_230703.txt'
ma_sim_f = os.path.join(data_root, ma_sim_f)
ma_sim_df = pd.read_csv(ma_sim_f, sep='\t', low_memory=False)

map_cid2prot_index = dict()
map_cid2na_index   = dict()
for i in range(len(ma_sim_df)):
    row = ma_sim_df.loc[i]
    map_cid2prot_index[ int(row['complex_id']) ] = str(row['protein_index'])
    map_cid2na_index[ int(row['complex_id']) ] = str(row['na_index'])

dir_sim = 'sim_matrix'
sim_prot_f = 'protein_seq_wt_mut_pident_230630.txt'
sim_ddna_f = 'dsDNA_ident_matrix.txt'
sim_drna_f = 'dsRNA_ident_matrix.txt'
sim_sdna_f = 'ssDNA_ident_matrix.txt'
sim_srna_f = 'ssRNA_ident_matrix.txt'

sim_prot_df = pd.read_table(os.path.join(data_root, dir_sim, sim_prot_f), index_col=0)

map_nc_type2df = dict()
map_nc_type2df['dsDNA'] = pd.read_table(os.path.join(data_root, dir_sim, sim_ddna_f), index_col=0)
map_nc_type2df['dsRNA'] = pd.read_table(os.path.join(data_root, dir_sim, sim_drna_f), index_col=0)
map_nc_type2df['ssDNA'] = pd.read_table(os.path.join(data_root, dir_sim, sim_sdna_f), index_col=0)
map_nc_type2df['ssRNA'] = pd.read_table(os.path.join(data_root, dir_sim, sim_srna_f), index_col=0)




# def func(data_root, dir_sim, fname, ):
#     f = os.path.join(data_root, dir_sim, fname)
#     df = pd.read_csv(f, sep='\t', low_memory=False)
    
#
# i: cluster_id -> unique complex
# j: cluster_id -> unique complex

def getDataByNcType(class_name, mapping):
    # DDNACluster
    if 'DDNACluster' in class_name:
        return 'dsDNA', mapping['dsDNA']
    elif 'DRNACluster' in class_name:
        return 'dsRNA', mapping['dsRNA']
    elif 'SDNACluster' in class_name:
        return 'ssDNA', mapping['ssDNA']
    elif 'SRNACluster' in class_name:
        return 'ssRNA', mapping['ssRNA']
    else:
        print("get base class name with a wrong NC class: {}".format(class_name))
        return None, None

###
for ci in all_cls_set:
    for cj in all_cls_set:
        cids_i = map_cls2cid[ci]
        cids_j = map_cls2cid[cj]
        # clu_i ==> prot_clu_i, nc_clu_i
        # clu_j ==> prot_clu_j, nc_clu_j
        nc_type_i, nc_sim_data_i = getDataByNcType(ci, map_nc_type2df)
        nc_type_j, nc_sim_data_j = getDataByNcType(cj, map_nc_type2df)
        
        mP_sim = getMeanSim(sim_prot_df, map_cid2prot_index, cids_i, cids_j)
        
        mN_dist = None
        if nc_type_i == nc_type_j:
            
            mN_sim = getMeanSim(map_nc_type2df[nc_type_i], 
                                map_cid2na_index, 
                                cids_i, cids_j)
        else:
            # different type indicates largest distance 
            mN_dist = np.ones([len(cids_i), len(cids_j)], dtype=np.float64)

KeyError: "None of [Index(['dsDNA97'], dtype='object')] are in the [index]"

In [160]:
f = 'seq_dg_230703.txt'
f = os.path.join(data_root, f)
print(f)
df = pd.read_csv(f, sep='\t', low_memory=False)
# df[df['newnafea_na_job'] == 'new_DNA_3110']
df[df['na_index'].str.contains('3110')]

../data/_datasets/cluster_res/seq_dg_230703.txt


,complex_id,dataset_id,UniProt,protein_sequence,nucleotide_sequence,nucleic_acid_mutation,protein_mutation,nucleic_acid_type,nucleic_acid_type_new,Kd,dG,pH,Tm,Method,pmid,data_source,protein_na_pair,multiple_dg,max_min,max_min_judge,dG_ori,pdb_id,len_p1,len_p2,protein_chain,na_chain,identity_protein_seq,identity_na_seq,UniProt_type,protein_sequence_len,wt_af2_file_download,mo_pdb_chain,bs_score,nucleotide_sequence_one,nucleotide_sequence_two,nucleotide_sequence_two_reverse,align_score,len_nucleotide_sequence_one,len_nucleotide_sequence_two,coverage_nucleotide_sequence_one,coverage_nucleotide_sequence_two,coverage_mean_nucleotide_sequence,protein_index,cluster_protein_50_index,cluster_protein_50_number,cluster_protein_50_member,na_index,cluster_na_50_index,cluster_na_50_number,cluster_na_50_member,protein_na_pair_index,protein_50_na_50_member,protein_50_na_50_number,cluster_protein_na_pair_index,dataset_id_pdb_id_protein_index,protein_species,na_jobid,nucleotide_sequence_fornew_na_fea,newnafea_na_job,AF2_filname,AF2_protein_sequence_len,pLDDT_median,protein_50_na_50_member_Li_clustertype,protein_50_na_50_number_Li_clustertype,cluster_protein_na_pair_index_Li_clustertype,mutation_type,protein_sequence_number,protein_sequence_wild,complex_type,dist_index


In [ ]:
df

In [131]:
getMeanSim(nc_sim_data_i, map_cid2na_index, cids_i, cids_j)

KeyError: "None of [Index(['dsDNA97'], dtype='object')] are in the [index]"

In [137]:
nc_type_i, nc_type_j

('dsDNA', 'dsDNA')

In [138]:
cids_i, cids_j

(array([10263]), array([10263]))

In [141]:
# cid -> protein_index -> sim_mat.index
indices_a = np.array([map_cid2na_index[c] for c in cids_i])
indices_b = np.array([map_cid2na_index[c] for c in cids_j])
print(indices_a)
print(indices_b)
sim = map_nc_type2df[nc_type_i].loc[indices_a, indices_b] # (num_a, num_b)
sim

['dsDNA97']
['dsDNA97']


KeyError: "None of [Index(['dsDNA97'], dtype='object')] are in the [index]"

In [140]:
map_nc_type2df[nc_type_i]

,dsdnanew_DNA_2390,dsdnanew_DNA_3154,dsdnanew_DNA_2938,dsdnanew_DNA_1735,dsdnanew_DNA_2628,dsdnanew_DNA_1894,dsdnanew_DNA_2610,dsdnanew_DNA_2746,dsdnanew_DNA_3259,dsdnanew_DNA_3057,dsdnanew_DNA_3013,dsdnanew_DNA_2675,dsdnanew_DNA_1950,dsdnanew_DNA_2588,dsdnanew_DNA_2868,dsdnanew_DNA_1683,dsdnanew_DNA_3107,dsdnanew_DNA_2994,dsdnanew_DNA_2295,dsdnanew_DNA_1653,dsdnanew_DNA_2472,dsdnanew_DNA_3095,dsdnanew_DNA_3064,dsdnanew_DNA_2544,dsdnanew_DNA_3344,dsdnanew_DNA_2138,dsdnanew_DNA_3372,dsdnanew_DNA_2575,dsdnanew_DNA_1942,dsdnanew_DNA_1869,dsdnanew_DNA_2010,dsdnanew_DNA_1810,dsdnanew_DNA_2729,dsdnanew_DNA_2124,dsdnanew_DNA_2678,dsdnanew_DNA_2916,dsdnanew_DNA_2220,dsdnanew_DNA_2971,dsdnanew_DNA_2793,dsdnanew_DNA_1972,dsdnanew_DNA_2987,dsdnanew_DNA_2881,dsdnanew_DNA_2551,dsdnanew_DNA_3308,dsdnanew_DNA_2582,dsdnanew_DNA_3373,dsdnanew_DNA_3128,dsdnanew_DNA_2535,dsdnanew_DNA_1755,dsdnanew_DNA_2432,...,dsdnanew_DNA_2815,dsdnanew_DNA_2648,dsdnanew_DNA_1806,dsdnanew_DNA_2539,dsdnanew_DNA_2062,dsdnanew_DNA_3032,dsdnanew_DNA_1792,dsdnanew_DNA_2896,dsdnanew_DNA_2755,dsdnanew_DNA_3179,dsdnanew_DNA_2843,dsdnanew_DNA_2023,dsdnanew_DNA_1809,dsdnanew_DNA_1762,dsdnanew_DNA_3010,dsdnanew_DNA_1743,dsdnanew_DNA_2853,dsdnanew_DNA_3022,dsdnanew_DNA_2330,dsdnanew_DNA_3132,dsdnanew_DNA_1640,dsdnanew_DNA_2421,dsdnanew_DNA_1629,dsdnanew_DNA_3368,dsdnanew_DNA_3063,dsdnanew_DNA_1927,dsdnanew_DNA_2844,dsdnanew_DNA_2199,dsdnanew_DNA_2457,dsdnanew_DNA_3009,dsdnanew_DNA_2806,dsdnanew_DNA_2152,dsdnanew_DNA_3171,dsdnanew_DNA_2744,dsdnanew_DNA_2906,dsdnanew_DNA_3325,dsdnanew_DNA_2694,dsdnanew_DNA_2328,dsdnanew_DNA_1991,dsdnanew_DNA_2608,dsdnanew_DNA_2004,dsdnanew_DNA_2855,dsdnanew_DNA_2500,dsdnanew_DNA_2828,dsdnanew_DNA_2344,dsdnanew_DNA_3104,dsdnanew_DNA_2055,dsdnanew_DNA_2228,dsdnanew_DNA_2515,dsdnanew_DNA_2297
dsdnanew_DNA_2390,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
dsdnanew_DNA_3154,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
dsdnanew_DNA_2938,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.969,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.969,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
dsdnanew_DNA_1735,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
dsdnanew_DNA_2628,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...